# Load Model

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('./runs/detect/person/weights/last.pt')

# Train Model

In [3]:
results = model.train(data='data.yaml',epochs=50, imgsz=640, pretrained=True, lr0=0.000001, lrf=0.0000001, batch=4, name='person',min_memory=True, resume=True)

Ultralytics YOLOv8.0.40  Python-3.9.16 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
yolo\engine\trainer: task=detect, mode=train, model=runs\detect\person\weights\last.pt, data=data.yaml, epochs=50, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=person, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=True, overlap_mask=True, mask_ratio=4, dropout=False, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript

In [4]:
model = YOLO('./runs/detect/person/weights/best.pt')

# Predict on image

In [7]:
model.predict('test.webp',save=True,conf=0.4)


image 1/1 D:\Object Detection Project\person\test.webp: 448x640 7 persons, 106.0ms
Speed: 3.0ms pre-process, 106.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 6])
 dtype: torch.float32
  + tensor([[6.90000e+01, 4.80000e+01, 9.19000e+02, 1.04700e+03, 8.96142e-01, 0.00000e+00],
         [6.67000e+02, 1.63000e+02, 1.45300e+03, 1.04200e+03, 8.89714e-01, 0.00000e+00],
         [1.38500e+03, 5.73000e+02, 1.47700e+03, 6.84000e+02, 7.74518e-01, 0.00000e+00],
         [7.41000e+02, 4.59000e+02, 8.43000e+02, 7.00000e+02, 7.13142e-01, 0.00000e+00],
         [6.73000e+02, 4.50000e+02, 7.60000e+02, 6.89000e+02, 4.86544e-01, 0.00000e+00],
         [1.60000e+01, 4.63000e+02, 1.08000e+02, 7.16000e+02, 4.82505e-01, 0.00000e+00],
         [1.52000e+02, 7.90000e+01, 2.38000e+02, 1.83000e+02, 4.41385e-01, 0.00000e+00]], device='cuda:0')]

In [12]:
from PIL import Image

img = Image.open('./runs/detect/predict/test.webp')

img.show()

# Deployment on video

In [4]:
import random

import cv2
import numpy as np
from ultralytics import YOLO

class_list = ['person']

# print(class_list)

# Generate random colors for class list
detection_colors = []
for i in range(len(class_list)):
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    detection_colors.append((b, g, r))

# load a pretrained YOLOv8n model
model = YOLO('./runs/detect/person/weights/best.pt')
# model(iou=0.7, conf=0.5)

# Vals to resize video frames | small frame optimise the run
frame_wid = 640
frame_hyt = 480

# cap = cv2.VideoCapture(1)
cap = cv2.VideoCapture("street.mp4")

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    #  resize the frame | small frame optimise the run
    # frame = cv2.resize(frame, (frame_wid, frame_hyt))

    # Predict on image
    detect_params = model.predict(source=[frame], conf=0.50, save=False, iou=0.70)

    # Convert tensor array to numpy
    DP = detect_params[0].cpu().numpy()
    print(DP)

    if len(DP) != 0:
        for i in range(len(detect_params[0])):
            print(i)

            boxes = detect_params[0].boxes
            box = boxes[i]  # returns one box
            clsID = box.cls.cpu().numpy()[0]
            conf = box.conf.cpu().numpy()[0]
            bb = box.xyxy.cpu().numpy()[0]

            cv2.rectangle(
                frame,
                (int(bb[0]), int(bb[1])),
                (int(bb[2]), int(bb[3])),
                detection_colors[int(clsID)],
                3,
            )

            # Display class name and confidence
            font = cv2.FONT_HERSHEY_COMPLEX
            cv2.putText(
                frame,
                class_list[int(clsID)] + " " + str(round(conf, 2)) + "%",
                (int(bb[0]), int(bb[1]) - 10),
                font,
                1,
                (255, 255, 255),
                2,
            )

    # Display the resulting frame
    cv2.imshow("ObjectDetection", frame)

    # Terminate run when "Q" pressed
    if cv2.waitKey(1) == ord("q"):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.40  Python-3.9.16 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

0: 384x640 12 persons, 10.0ms
Speed: 0.0ms pre-process, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 persons, 12.6ms
Speed: 1.0ms pre-process, 12.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 12.0ms
Speed: 0.0ms pre-process, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


[[       1526         330        1641         681     0.89439           0]
 [        609         168         972        1074     0.85528           0]
 [          0         301         204        1070     0.84629           0]
 [        360         281         651        1072     0.83196           0]
 [       1639         327        1737         666     0.83118           0]
 [        997         318        1121         836     0.78973           0]
 [       1741         338        1859         740     0.78799           0]
 [       1196         283        1356         731     0.78277           0]
 [       1328         309        1426         689     0.77445           0]
 [       1813         281        1919         785     0.67509           0]
 [       1466         330        1550         644     0.57794           0]
 [       1292         337        1374         719     0.50073           0]]
0
1
2
3
4
5
6
7
8
9
10
11
[[        342         276         646        1072     0.89951           0


0: 384x640 13 persons, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 11.0ms
Speed: 1.0ms pre-process, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 8.0ms
Speed: 1.0ms pre-process, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 14 persons, 11.0ms
Speed: 1.0ms pre-process, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 10.0ms
Speed: 0.0ms pre-process, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 9.0ms
Speed: 0.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



[[        554         157         864        1074     0.90721           0]
 [       1547         328        1658         675     0.88275           0]
 [       1646         322        1754         663     0.88213           0]
 [        281         256         603        1070      0.8813           0]
 [       1767         324        1875         737     0.83326           0]
 [          0         307         182        1070      0.8277           0]
 [       1001         318        1125         834     0.80059           0]
 [       1337         302        1433         689     0.80025           0]
 [       1200         303        1352         729     0.71758           0]
 [       1469         329        1560         642      0.6966           0]
 [       1843         323        1919         786     0.65473           0]
 [        836         382         980         826     0.54356           0]
 [       1287         329        1381         714     0.53072           0]]
0
1
2
3
4
5
6
7
8
9
10
1

0: 384x640 11 persons, 9.0ms
Speed: 0.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 10.0ms
Speed: 1.0ms pre-process, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 10.0ms
Speed: 1.0ms pre-process, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 9.0ms
Speed: 0.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



...

[[        429         353         749        1071     0.90154           0]
 [         85         118         318        1070     0.88742           0]
 [       1711         702        1772         911     0.84641           0]
 [        883         582        1019         952     0.84286           0]
 [       1214         504        1424        1019     0.83733           0]
 [       1729         499        1918        1036     0.82967           0]
 [       1524         493        1697        1020     0.81324           0]
 [       1466         632        1544         919     0.77824           0]
 [        682         557         806         977     0.77605           0]
 [          0           3         209        1052     0.76828           0]
 [        805         575         898         937      0.7449           0]
 [       1059         561        1137         948     0.67453           0]
 [       1157         684        1221         893     0.64047           0]
 [       1001         626

0: 384x640 15 persons, 12.0ms
Speed: 1.0ms pre-process, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 persons, 13.0ms
Speed: 0.0ms pre-process, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 persons, 13.0ms
Speed: 1.0ms pre-process, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 16 persons, 13.0ms
Speed: 1.0ms pre-process, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 12 persons, 12.0ms
Speed: 1.0ms pre-process, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 13.0ms
Speed: 1.0ms pre-process, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



[[       1536         505        1696        1019     0.91979           0]
 [         92         119         321        1070     0.88905           0]
 [        436         350         744        1072     0.88816           0]
 [       1706         501        1889        1022     0.84001           0]
 [       1233         519        1438        1019     0.81889           0]
 [       1051         563        1140         964     0.79186           0]
 [       1398         570        1508         953     0.78655           0]
 [          0           3         212        1058     0.78162           0]
 [        667         528         808         983     0.76244           0]
 [         55          57         225        1074     0.71565           0]
 [        809         578         903         941     0.71328           0]
 [       1169         681        1227         896     0.65146           0]
 [        902         584        1018         953     0.63359           0]
 [       1481         626

0: 384x640 14 persons, 13.0ms
Speed: 0.0ms pre-process, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 13 persons, 13.0ms
Speed: 1.0ms pre-process, 13.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


[[       1701         508        1874        1016     0.92612           0]
 [        435         345         737        1071     0.89382           0]
 [       1532         509        1690        1012     0.88864           0]
 [         93         119         327        1069     0.87445           0]
 [       1411         564        1530         955     0.86339           0]
 [       1247         522        1435        1013     0.85975           0]
 [       1055         566        1149         966     0.81595           0]
 [          0           3         219        1055     0.77081           0]
 [        669         504         814         980     0.75977           0]
 [        811         581         907         946     0.68135           0]
 [       1173         686        1250         900     0.67657           0]
 [        925         590        1017         815     0.67411           0]
 [        993         635        1064         818     0.58296           0]
 [        993         637